# Импорт библиотек

In [1]:
import numpy as np
import os
import PIL
import pathlib
import tensorflow as tf
from tensorflow import keras
from PIL import Image, ImageFilter
from tqdm import tqdm

#tf.compat.v1.set_random_seed(290)
# детерминирование случайных величин
tf.random.set_seed(290)
np.random.seed(290)

### Загрузка датасета

In [2]:
# скачать датасет
# закомменть после того как скачаешь
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True, cache_dir=os.getcwd())
data_dir = pathlib.Path(archive).with_suffix('')

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 24s 0us/step


### Разметка данных (изначально в датасете нет размытых изображений)

In [3]:
# тоотношения тренировочной, валидационной и тестовой выборки
TRAIN_PERCENT = 0.60 
VAL_PERCENT = 0.20
TEST_PERCENT = 1 - TRAIN_PERCENT - VAL_PERCENT

if (TRAIN_PERCENT + VAL_PERCENT + TEST_PERCENT) != 1.:
    raise ValueError('Сумма процентов должна быть равна 1')

# размеры входного изображения для нейросети 
# к которому будут преобразованы все фото
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

# папка в которую загрузится датасет flower_photos от tensorflow.org
PATH_TO_UNPREPARED_DATASET = 'datasets/flower_photos.tgz/flower_photos'
# папка в которой будет храниться готовый датасет после предобработки
PATH_TO_SAVE_DATASET = 'labeled_dataset'

# словарь содержащий пути к фото для разных выборок
dataset = {'train':[],
           'val':[],
           'test':[]}

In [4]:
# распределяет все фото по тренировочному, валидационному и тестовому датасету
# нужно чтобы представители каждого класса из скачанного датасета присутствовали в 
# каждой выборке
for tup in os.walk(PATH_TO_UNPREPARED_DATASET):
    # tup - это кортеж 3 элементов
    # tup[0] - относительный путь до обрабатываемой папки
    # tup[1] - список всех папок в ней
    # tup[2] - список всех файлов в ней

    # если в папке есть изображения типа jpg
    if [filename for filename in tup[2] if filename.endswith('.jpg')]:
        print(f'Путь к фоторафиям: {tup[0]}. Кол-во фото в папке: {len(tup[2])}')
        # получить относительный путь ко всем фото в папке
        temp_arr = np.array([pathlib.Path(tup[0])/filename for filename in tup[2]])
        # разделить список путей к фото на 3 части для каждой выборки
        temp_train, temp_validate, temp_test = np.split(temp_arr, 
                                                        [int(temp_arr.shape[0]*TRAIN_PERCENT), 
                                                         int(temp_arr.shape[0]*(TRAIN_PERCENT+VAL_PERCENT))])
        # add ndarray to list of each dataset part
        # like dataset['train'] = list(np.ndarray(1,2,3), np.ndarray(4,5,6))
        dataset['train'].append(temp_train)
        dataset['val'].append(temp_validate)
        dataset['test'].append(temp_test)

# concatenate each numpy ndarray in each dataset part to one ndarray
for key, value in dataset.items():
    dataset[key] = np.concatenate(value)

datasets/flower_photos.tgz/flower_photos\daisy [] 633 items in current path
datasets/flower_photos.tgz/flower_photos\dandelion [] 898 items in current path
datasets/flower_photos.tgz/flower_photos\roses [] 641 items in current path
datasets/flower_photos.tgz/flower_photos\sunflowers [] 699 items in current path
datasets/flower_photos.tgz/flower_photos\tulips [] 799 items in current path


In [5]:
# print dataset info
for key, value in dataset.items():
    print('|'*8,f'{key} выборка','|'*8)
    print(f'\tКол-во наблюдений: {value.shape[0]}')
    print(f'\Пример путей к фото (первые два):',*value[:2],sep='\n\t')
    print()

|||||||| train part ||||||||
	Total items: 2199
	Example (first 2 items):
	datasets\flower_photos.tgz\flower_photos\daisy\100080576_f52e8ee070_n.jpg
	datasets\flower_photos.tgz\flower_photos\daisy\10140303196_b88d3d6cec.jpg

|||||||| val part ||||||||
	Total items: 735
	Example (first 2 items):
	datasets\flower_photos.tgz\flower_photos\daisy\4496202781_1d8e776ff5_n.jpg
	datasets\flower_photos.tgz\flower_photos\daisy\450128527_fd35742d44.jpg

|||||||| test part ||||||||
	Total items: 736
	Example (first 2 items):
	datasets\flower_photos.tgz\flower_photos\daisy\5948835387_5a98d39eff_m.jpg
	datasets\flower_photos.tgz\flower_photos\daisy\5973488341_50bdf6cee3_n.jpg



### Обработка размеченных данных (применение размытия) и помощение в новую папку

In [14]:
# blur labeled images and put all images in new folders
for key, value in dataset.items():
    # create path to save new photo
    path_to_save_blur =  pathlib.Path(PATH_TO_SAVE_DATASET)/key/'blur'
    if not os.path.exists(path_to_save_blur):
        os.makedirs(path_to_save_blur)
        
    path_to_save_sharp =  pathlib.Path(PATH_TO_SAVE_DATASET)/key/'sharp'
    if not os.path.exists(path_to_save_sharp):
        os.makedirs(path_to_save_sharp)

    for image_path in tqdm(value, desc=f'Подготовка данных для {key} выборки'):
        filename = image_path.name
        
        img = keras.utils.load_img(image_path)
        img = keras.utils.img_to_array(img)
        img = keras.layers.Resizing(IMAGE_HEIGHT, IMAGE_WIDTH)(img)
        img = keras.utils.array_to_img(img)
        img.save(path_to_save_sharp / filename)

        filename = "blur_"+filename
        
        img = img.filter(ImageFilter.BLUR)
        img.save(path_to_save_blur / filename)
        
print('Подготовка завершена')

Prepare test dataset part: 100%|████████████████████████████████████████████████████| 736/736 [00:05<00:00, 123.33it/s]
